In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

<h4>Данные</h4>

<p>UNESCO Bulk Data Download Service can be accessed here: <a href>https://uis.unesco.org/bdds</a> 
<br> Labels list is analyzed and students inbound indicators are extracted into distinct csv file named <i>OPRI_student_labels.csv</i>
<br>UNESCO Data Service is also availabla at https://data.uis.unesco.org/#
<p>UNESCO UIS DataSet for 2017 and 2022 lacks data from USA, Mexico, Netherlands, Iran, Kazakhstan and some other countries. They are taken from the following sources:
<br> USA, 2022 - <a href>https://opendoorsdata.org</a> (by source countries)
<br>China, 2022 - http://www.moe.gov.cn/jyb_xwfb/gzdt_gzdt/s5987/201904/t20190412_377692.html taken pre-pandemic for distinct countries, total from http://en.moe.gov.cn/documents/statistics/2022/national/202401/t20240110_1099530.html for 2022
<br>China, 2017 - are from 2018 https://docs.aiddata.org/reports/corridors-of-power.html#figure4
<br>Philippines
<br>Israel
<br> Netherlands, 2022 - https://www.nuffic.nl/sites/default/files/2023-06/incoming-degree-mobility-in-dutch-higher-education-2022-2023.pdf (total and by source countries where provided)
<br> Mexico, 2022 - https://www.iie.org/research-initiatives/project-atlas/explore-data/mexico-2/ (total inbound students)
<br> Iran, 2022 - https://tehrantimes.com/news/483478/Students-from-91-countries-studying-in-Iranian-universities (total inbound students)
<br> Kazakhstan, 2022 - https://enic-kazakhstan.edu.kz/uploads/additional_files_items/193/file/tadzhikistan.pdf?cache=1691397957, https://enic-kazakhstan.edu.kz/uploads/additional_files_items/156/file/mezhdunarodnoe-sotrudnichestvo-sng.pdf?cache=1677496628, https://enic-kazakhstan.edu.kz/uploads/additional_files_items/138/file/3-1-monitoring-akademicheskoy-mobilnosti-2022-rus.pdf?cache=1672983807 (total and by source countries where provided)
<br> Egypt, 2017 - https://enterprise.press/stories/2019/12/16/how-egypt-is-positioning-itself-as-an-educational-hub-for-international-students-8512/ (total inbound students)
<br> Thailand, 2017 - https://apheit.bu.ac.th/jounal/Inter-vol8-1/นานาชาติ_บทความวิจัย_2.pdf (total inbound students)


In [4]:
country_names = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_COUNTRY.csv")
print(country_names.columns)
country_names.head(2)

Index(['COUNTRY_ID', 'COUNTRY_NAME_EN'], dtype='object')


,COUNTRY_ID,COUNTRY_NAME_EN
0,AFG,Afghanistan
1,ALB,Albania


In [5]:
labels = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_students_label.csv")
print(labels.columns)
labels.head(2)

Index(['INDICATOR_ID', 'INDICATOR_LABEL_EN'], dtype='object')


,INDICATOR_ID,INDICATOR_LABEL_EN
0,26421,"Africa: Students from Algeria, both sexes (num..."
1,26422,"Africa: Students from Angola, both sexes (number)"


In [6]:
national = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_DATA_NATIONAL.csv")
print(national.columns)
national.head(2)

C:\Users\yuliy\AppData\Local\Temp\ipykernel_20200\3015259283.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  national = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_DATA_NATIONAL.csv")


Index(['indicator_id', 'country_id', 'year', 'value', 'magnitude',
       'qualifier'],
      dtype='object')


,indicator_id,country_id,year,value,magnitude,qualifier
0,10,ABW,1998,0.0,NaN,NaN
1,10,ABW,1999,0.0,NaN,NaN


In [7]:
#Add USA inbound students
usa_add_long = pd.read_csv("usa_inbound.csv")
usa_add_long['to'] = 'United States of America'
usa_add_long['year'] = '2022'
usa_add_long = usa_add_long.rename(columns={'unesco':'from'})
usa_add_long = usa_add_long[['from','to','year','2022']] #берем только 2022
usa_add_long.columns = ['from','to','year','value']
usa_add_long = usa_add_long[usa_add_long['from']!='#Н/Д']
usa_add_long['value'] = usa_add_long['value'].fillna(0).astype(int)
usa_add_long['country_id'] = 'USA'
print(usa_add_long.value.sum())
usa_add_long.head(2)

1035161


,from,to,year,value,country_id
0,Afghanistan,United States of America,2022,682,USA
1,Albania,United States of America,2022,1087,USA


In [8]:
#Add Kazakhstan inbound students
kaz_add_long = pd.read_csv("kazakhstan_inbound.csv")
kaz_add_long['year'] = kaz_add_long['year'].astype(str)
kaz_add_long['country_id'] = 'KAZ'
kaz_add_long.head(2)

,from,to,year,value,country_id
0,Uzbekistan,Kazakhstan,2022,9571.0,KAZ
1,Tajikistan,Kazakhstan,2022,361.0,KAZ


In [9]:
#Addd Netherlands inbound students
nld_add_long = pd.read_csv("nld_inbound.csv")
nld_add_long  = nld_add_long.dropna()
nld_add_long['year'] = nld_add_long['year'].astype(str)
nld_add_long['country_id'] = 'NLD'
nld_add_long.head(2)

,from,to,year,value,country_id
18,Belgium,Netherlands,2022,4780.0,NLD
29,Bulgaria,Netherlands,2022,5112.0,NLD


In [10]:
china_add_long = pd.read_csv("china_inbound.csv")
china_add_long  = china_add_long.dropna()
china_add_long['year'] = china_add_long['year'].astype(str)
china_add_long['country_id'] = 'CHN'
china_add_long.head(2)

,from,to,year,value,country_id
0,India,China,2017,23198,CHN
1,India,China,2022,23198,CHN


inbound отчеты - обработка и добавление отсутствующих стран

In [11]:
#Extract labels and countru names from UIS dataset
from_to_labels = labels[~labels['INDICATOR_ID'].isin([26637,26638,43188])]
from_to_labels['from'] = from_to_labels['INDICATOR_LABEL_EN'].apply(lambda x:x.split('from ')[1].split(', both sexes (number)')[0].replace('the ',''))
from_to_labels.head()

,INDICATOR_ID,INDICATOR_LABEL_EN,from
0,26421,"Africa: Students from Algeria, both sexes (num...",Algeria
1,26422,"Africa: Students from Angola, both sexes (number)",Angola
2,26423,"Africa: Students from Benin, both sexes (number)",Benin
3,26424,"Africa: Students from Botswana, both sexes (nu...",Botswana
4,26425,"Africa: Students from Burkina Faso, both sexes...",Burkina Faso


In [12]:
total_from_to = national[national['indicator_id'].isin(from_to_labels['INDICATOR_ID'].unique())]
total_from_to = pd.merge(total_from_to,from_to_labels,left_on='indicator_id',right_on='INDICATOR_ID',how='left')
total_from_to = pd.merge(total_from_to,country_names,left_on='country_id',right_on='COUNTRY_ID',how='left')
total_from_to = total_from_to.rename(columns={'COUNTRY_NAME_EN':'to'})
total_from_to =total_from_to[['indicator_id','country_id','year','value','from','to']]
total_from_to['value'] = total_from_to['value'].astype(int)
total_from_to['year'] = total_from_to['year'].astype(str)
print(total_from_to.shape)
total_from_to.head()

(463913, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0,Algeria,Aruba
1,26421,ABW,2004,0,Algeria,Aruba
2,26421,ABW,2006,0,Algeria,Aruba
3,26421,ABW,2009,0,Algeria,Aruba
4,26421,ABW,2010,0,Algeria,Aruba


In [13]:
canada_in = total_from_to[total_from_to['to'] == 'United States of America']
canada_in
#не хватает 2022, добавляем из файла с фильтром по году

,indicator_id,country_id,year,value,from,to
2118,26421,USA,1999,202,Algeria,United States of America
2119,26421,USA,2005,149,Algeria,United States of America
2120,26421,USA,2013,168,Algeria,United States of America
2121,26421,USA,2014,164,Algeria,United States of America
2122,26421,USA,2015,162,Algeria,United States of America
...,...,...,...,...,...,...
463840,26652,USA,2016,118,Montenegro,United States of America
463841,26652,USA,2017,108,Montenegro,United States of America
463842,26652,USA,2018,116,Montenegro,United States of America
463843,26652,USA,2020,120,Montenegro,United States of America


In [14]:
total_from_to[total_from_to['to'] == 'Netherlands']
#не хватает 2022, добавляем из файла

,indicator_id,country_id,year,value,from,to
1544,26421,NLD,1999,17,Algeria,Netherlands
1545,26421,NLD,2000,19,Algeria,Netherlands
1546,26421,NLD,2001,19,Algeria,Netherlands
1547,26421,NLD,2002,21,Algeria,Netherlands
1548,26421,NLD,2003,18,Algeria,Netherlands
...,...,...,...,...,...,...
463434,26652,NLD,2015,5,Montenegro,Netherlands
463435,26652,NLD,2016,7,Montenegro,Netherlands
463436,26652,NLD,2017,13,Montenegro,Netherlands
463437,26652,NLD,2018,15,Montenegro,Netherlands


In [15]:
total_from_to[total_from_to['to'] == 'Kazakhstan']
#не хватает 2021, 2022, добавляем из файла

,indicator_id,country_id,year,value,from,to
1059,26421,KAZ,1999,0,Algeria,Kazakhstan
1060,26421,KAZ,2000,0,Algeria,Kazakhstan
1061,26421,KAZ,2001,1,Algeria,Kazakhstan
1062,26421,KAZ,2003,0,Algeria,Kazakhstan
1063,26421,KAZ,2004,0,Algeria,Kazakhstan
...,...,...,...,...,...,...
463070,26652,KAZ,2016,0,Montenegro,Kazakhstan
463071,26652,KAZ,2017,0,Montenegro,Kazakhstan
463072,26652,KAZ,2018,0,Montenegro,Kazakhstan
463073,26652,KAZ,2019,0,Montenegro,Kazakhstan


In [16]:
#delete empty 2022 rows for the Netherlands

# total_from_to = total_from_to[~((total_from_to['to'] == 'United States of America') & (total_from_to['year'] == '2022'))]
total_from_to = total_from_to[~((total_from_to['to']=='Netherlands') & (total_from_to['year']=='2022'))]
# total_from_to = total_from_to[~((total_from_to['to']=='Kazakhstan') & (total_from_to['year']=='2022'))]
# total_from_to = total_from_to[~((total_from_to['to']=='China') & (total_from_to['year']=='2022'))]
# total_from_to = total_from_to[~((total_from_to['to']=='China') & (total_from_to['year']=='2017'))]
print(total_from_to.shape)
total_from_to.head(2)

(463912, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0,Algeria,Aruba
1,26421,ABW,2004,0,Algeria,Aruba


In [18]:
canada_in = total_from_to[total_from_to['to'] == 'United States of America']
canada_in

,indicator_id,country_id,year,value,from,to
2118,26421,USA,1999,202,Algeria,United States of America
2119,26421,USA,2005,149,Algeria,United States of America
2120,26421,USA,2013,168,Algeria,United States of America
2121,26421,USA,2014,164,Algeria,United States of America
2122,26421,USA,2015,162,Algeria,United States of America
...,...,...,...,...,...,...
463840,26652,USA,2016,118,Montenegro,United States of America
463841,26652,USA,2017,108,Montenegro,United States of America
463842,26652,USA,2018,116,Montenegro,United States of America
463843,26652,USA,2020,120,Montenegro,United States of America


In [19]:
print(total_from_to.shape)
#Add absent countries into df
total_from_to = pd.concat([total_from_to,usa_add_long,kaz_add_long,nld_add_long,china_add_long])
print(total_from_to.shape)
# total_from_to['value'] = total_from_to['value'].astype(int)
total_from_to.head(2)

(463912, 6)
(464215, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0.0,Algeria,Aruba
1,26421,ABW,2004,0.0,Algeria,Aruba


In [ ]:
#replace long country names by short to make charts cleaner
total_from_to.loc[total_from_to['from']=='Iran (Islamic Republic of)','from']='Iran'
total_from_to.loc[total_from_to['from']=='Islamic Republic of Iran','from']='Iran'
total_from_to.loc[total_from_to['from']=='United Kingdom of Great Britain and Northern Ireland','from']='United Kingdom'
total_from_to.loc[total_from_to['from']=='China, Hong Kong Special Administrative Region','from']='Hong Kong'
total_from_to.loc[total_from_to['from']=='Hong Kong, Special Administrative Region of China','from']='Hong Kong'
total_from_to.loc[total_from_to['from']=='China, Macao Special Administrative Region','from']='Macao'
total_from_to.loc[total_from_to['from']=='Macao, Special Administrative Region of China','from']='Macao'
total_from_to.loc[total_from_to['from']=='United States of America','from']='USA'
total_from_to.loc[total_from_to['from']=='United States','from']='USA'
total_from_to.loc[total_from_to['from']=='United Republic of Tanzania','from']='Tanzania'
total_from_to.loc[total_from_to['from']=="Lao People's Democratic Republic",'from'] = 'Lao'
total_from_to.loc[total_from_to['from']=="Turks and Caicos Islands",'from'] = 'Turks and Caicos'
total_from_to.loc[total_from_to['from']=="Russian Federation",'from'] = 'Russia'
total_from_to.loc[total_from_to['from']=="Republic of Moldova",'from'] = 'Moldova'
total_from_to.loc[total_from_to['to']=='Iran (Islamic Republic of)','to']='Iran'
total_from_to.loc[total_from_to['to']=='Islamic Republic of Iran','to']='Iran'
total_from_to.loc[total_from_to['to']=='United Kingdom of Great Britain and Northern Ireland','to']='United Kingdom'
total_from_to.loc[total_from_to['to']=='China, Hong Kong Special Administrative Region','to']='Hong Kong'
total_from_to.loc[total_from_to['to']=='Hong Kong, Special Administrative Region of China','to']='Hong Kong'
total_from_to.loc[total_from_to['to']=='China, Macao Special Administrative Region','to']='Macao'
total_from_to.loc[total_from_to['to']=='Macao, Special Administrative Region of China','to']='Macao'
total_from_to.loc[total_from_to['to']=='United States of America','to']='USA'
total_from_to.loc[total_from_to['to']=='United States','to']='USA'
total_from_to.loc[total_from_to['to']=='United Republic of Tanzania','to']='Tanzania'
total_from_to.loc[total_from_to['to']=="Lao People's Democratic Republic",'to'] = 'Lao'
total_from_to.loc[total_from_to['to']=="Turks and Caicos Islands",'to'] = 'Turks and Caicos'
total_from_to.loc[total_from_to['to']=="Russian Federation",'to'] = 'Russia'
total_from_to.loc[total_from_to['to']=="Republic of Moldova",'to'] = 'Moldova'

International Education Market

In [21]:
total_years = total_from_to.groupby(['year','to'])['value'].sum().reset_index()
#raw data from Unesco and additional by-outbound-countries datasets. Data for country level are added or corrected later
#this df allows to see if we have data for a country in a distinct year
total_years[total_years['year'].isin(['2017','2022'])]

,year,to,value
2321,2017,Algeria,0.0
2322,2017,Andorra,182.0
2323,2017,Argentina,86665.0
2324,2017,Armenia,4138.0
2325,2017,Australia,375140.0
...,...,...,...
3013,2022,Turks and Caicos,183.0
3014,2022,USA,1035161.0
3015,2022,United Kingdom,674440.0
3016,2022,Uzbekistan,5502.0


In [22]:
country='Egypt'
df_country = total_years.pivot(index=['to'], columns='year', values='value').reset_index()
# df_country = df_country[df_country['to']==country]
df_country = df_country[['to','2017','2019','2020','2021','2022']]
df_country

year,to,2017,2019,2020,2021,2022
0,Afghanistan,NaN,NaN,0.0,NaN,NaN
1,Albania,NaN,2156.0,2146.0,1944.0,1731.0
2,Algeria,0.0,0.0,0.0,0.0,0.0
3,American Samoa,NaN,NaN,NaN,NaN,NaN
4,Andorra,182.0,258.0,286.0,282.0,NaN
...,...,...,...,...,...,...
200,Venezuela (Bolivarian Republic of),NaN,NaN,NaN,NaN,NaN
201,Viet Nam,4125.0,7248.0,8645.0,8369.0,7759.0
202,Yemen,NaN,NaN,NaN,NaN,NaN
203,Zambia,NaN,NaN,NaN,NaN,NaN


In [653]:
df_country.sum()

year
to      AfghanistanAlbaniaAlgeriaAmerican SamoaAndorra...
2017                                            4838081.0
2019                                            4075461.0
2020                                            5329383.0
2021                                            5241636.0
2022                                            6022009.0
dtype: object

In [ ]:
#не хватает крупных рынков - импорт
# Египет: 2017, 2022
# Иран: 2022
# Израиль: 2017, 2022
# панама: 2017
# Сингапур: 2017, 2022
# Philippines: 2017, 2022
# Thailand: 2017
# Togo: 2017,2022
# Ukraine: 2022
# Uruguay: 2017,2022
# Mexico:2022
# куба: 2022 - взять 2021
# Yemen
# Zambia
# Zimbabwe
#Indonesia 2022

<h4>Inbound by Countries</h4>

На основании отчетов inbound в разрезе страна-страна

In [23]:
countries = total_from_to[total_from_to['year'].isin(['2017','2022'])]
countries['year'] = countries['year'].astype(str)
countries = countries[countries['from']!=countries['to']]
countries = countries.groupby(['year','from','to'])['value'].sum().reset_index()
countries = countries.pivot(index=['from','to'], columns='year', values='value').reset_index()
countries = countries.fillna(0)
# countries = countries[countries['from']!='unknown countries']
countries['from'] = countries['from'].apply(lambda x:x.strip())
countries['to'] = countries['to'].apply(lambda x:x.strip())
countries.head()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_20200\3490173088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries['year'] = countries['year'].astype(str)


year,from,to,2017,2022
0,Afghanistan,Albania,0.0,0.0
1,Afghanistan,Andorra,0.0,0.0
2,Afghanistan,Argentina,15.0,16.0
3,Afghanistan,Armenia,0.0,0.0
4,Afghanistan,Australia,44.0,73.0


In [26]:
#Add total inbound students for absent countries where we don't have disctinct values by source countries
inbound = pd.merge(countries.groupby(['to'])['2017'].sum().reset_index(),
                 countries.groupby(['to'])['2022'].sum().reset_index(),
                 on = 'to',
                 how='left')
inbound.fillna(0,inplace=True)

new_data = [
    {'to': 'Egypt', '2017': 51162,'2022':69021} #2021 data https://www.statista.com/statistics/1193448/number-of-students-in-higher-education-in-egypt-by-nationality/
]
new_data_df = pd.DataFrame(new_data)
inbound = pd.concat([inbound,new_data_df])

inbound.loc[inbound['to']=='Thailand','2017']=43821
inbound.loc[inbound['to']=='Mexico','2022']=51659
inbound.loc[inbound['to']=='Iran','2022']=94406
inbound.loc[inbound['to']=='Ukraine','2022']=50870
inbound.loc[inbound['to']=='Cuba','2022']=7773
inbound.loc[inbound['to']=='Kazakhstan','2022']=26080 #add values by countries and total, as not every source is known
inbound.loc[inbound['to']=='China','2022']=253177
inbound.loc[inbound['to']=='China','2017']=489172
inbound.loc[inbound['to']=='Netherlands','2022']=122287
inbound.head()


,to,2017,2022
0,Albania,0.0,1731.0
1,Andorra,182.0,0.0
2,Argentina,86665.0,136259.0
3,Armenia,4138.0,6635.0
4,Australia,375140.0,369923.0


In [27]:
#Этап до удаления нулевых значений. Считаем сумму для каждого года и разницу сумм
world = pd.DataFrame({'2017':[inbound['2017'].sum()],'2022':[inbound['2022'].sum()]})
world['2022/2017,%'] = round(world['2022']/world['2017']*100,0).astype(int)-100
world['2022/2017,%'] = world['2022/2017,%'].apply(lambda x:'+'+str(x) if x>0 else str(x))
world


,2017,2022,"2022/2017,%"
0,5147503.0,6283374.0,+22


In [28]:

fig = go.Figure()
trace1 = go.Bar(x=world.index,
    y=world['2017'],
    name=f'2017',
    text=[f"{value/1000000:.1f}m" if value < 1000 else f"{value/1000000:.1f}m" for value in world['2017']],  # Value labels in thousands
    textposition='auto',
    textfont=dict(size=16),
    marker_color='blue',
    marker_line=dict(width=3, color=['red' if country == 'Russia' else 'blue' for country in world.index]) 
    )
trace2 = go.Bar(x=world.index,
    y=world['2022'],
    name=f'2022',
    text=[f"{value/1000000:.1f}m\n({world['2022/2017,%'].values[j]}%)" if value < 1000 
          else f"{value/1000000:.1f}m\n({world['2022/2017,%'].values[j]}%)" for j, value in enumerate(world['2022'])],  # Value labels in thousands
    textposition='auto',
    textfont=dict(size=16),
    marker_color='green',
    marker_line=dict(width=3, color=['red' if country == 'Russia' else 'green' for country in world.index]) 
    )

fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title='Входящий поток иностранных студентов по группам стран с 2017 по 2022 г.',
    # xaxis_title='Country',
    yaxis_title='Число студентов, тыс.',
    barmode='group',  # Group bars together
    height=400,
    width=600,
    plot_bgcolor='white'
)
# for i in range(1, num_rows + 1):
#     for j in range(1, num_cols + 1):
#         fig.update_xaxes(tickangle=45, row=i, col=j)
#         fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Число иностранных студентов в мире 2017 и 2022.png")
fig.show()
print(world)

        2017       2022 2022/2017,%
0  5147503.0  6283374.0         +22


In [29]:
#возвращаемся к страновому разрезу, удаляем нулевые строки, считаем динамику и рисуем чарт
inbound_no_zero = inbound[(inbound['2017'] != 0) & (inbound['2022'] != 0)]

inbound_no_zero['2022/2017,%'] = inbound_no_zero.apply(
    lambda row: int(round(row['2022'] / row['2017'] * 100, 0)) - 100
    if row['2022'] != 0 and row['2017'] != 0 else None,
    axis=1
)


# inbound_no_zero['2022/2017,%'] = inbound_no_zero['2022/2017,%'].apply(lambda x: int(x) if x is not None else None)

# # Format the output to add '+' sign for positive values
inbound_no_zero['2022/2017,%'] = inbound_no_zero['2022/2017,%'].apply(lambda x: '+' + str(x) if x > 0 else str(x))

#shorten the countries list
inbound_no_zero = inbound_no_zero[inbound_no_zero['2022']>5000]

#And assign groups to make understandable charts
def group(x):
    if x<20000:
        group_name = '5-20 тыс.'
    elif 20000<=x<50000:
        group_name = '20-50 тыс.'
    elif 50000<=x<100000:
        group_name = '50-100 тыс.'
    else:
        group_name = 'Больше 100 тыс.'
    return group_name
inbound_no_zero['group'] = inbound_no_zero['2022'].apply(lambda x: group(x))
inbound_no_zero.set_index('to', inplace=True)
inbound_no_zero = inbound_no_zero.sort_values(by='2022',ascending=False)
inbound_no_zero.head()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_20200\1757706485.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yuliy\AppData\Local\Temp\ipykernel_20200\1757706485.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,2017,2022,"2022/2017,%",group
to,,,,
USA,984170.0,1035161.0,+5,Больше 100 тыс.
United Kingdom,435389.0,674440.0,+55,Больше 100 тыс.
Germany,241497.0,374226.0,+55,Больше 100 тыс.
Australia,375140.0,369923.0,-1,Больше 100 тыс.
Canada,205816.0,336504.0,+63,Больше 100 тыс.


Inbound students

In [30]:
n_cols = 2  # Number of columns in subplot grid
# unique_groups  = diff['group'].unique()
unique_groups = ['Больше 100 тыс.', '50-100 тыс.', '20-50 тыс.','5-20 тыс.']
num_plots = len(unique_groups)
num_cols = 2  # Maximum of 2 columns
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=unique_groups)

for i, group in enumerate(unique_groups):
    filtered_df = inbound_no_zero[inbound_no_zero['group'] == group]
    row_index = i // n_cols + 1
    col_index = i % n_cols + 1
    
    trace1 = go.Bar(x=filtered_df.index,
        y=filtered_df['2017'],
        name=f'2017 - Group {group}',
        text=[f"{value/1000:.1f}k" if value < 1000 else f"{int(value/1000)}k" for value in filtered_df['2017']],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=16),
        marker_color='blue',
        marker_line=dict(width=3, color=['red' if country == 'Russia' else 'blue' for country in filtered_df.index]) 
        )
    trace2 = go.Bar(x=filtered_df.index,
        y=filtered_df['2022'],
        name=f'2022 - Group {group}',
        text=[f"{value/1000:.1f}k\n({filtered_df['2022/2017,%'].values[j]}%)" if value < 1000 else f"{int(value/1000)}k\n({filtered_df['2022/2017,%'].values[j]}%)" for j, value in enumerate(filtered_df['2022'])],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=16),
        marker_color='green',
        marker_line=dict(width=3, color=['red' if country == 'Russia' else 'green' for country in filtered_df.index]) 
        )
    
    fig.add_trace(trace1, row=row_index, col=col_index)
    fig.add_trace(trace2, row=row_index, col=col_index)
        
fig.update_layout(
title='Входящий поток иностранных студентов по группам стран с 2017 по 2022 г.',
# xaxis_title='Country',
yaxis_title='Число студентов, тыс.',
barmode='group',  # Group bars together
height=1400,
width=2600,
plot_bgcolor='white'
)
for i in range(1, num_rows + 1):
    for j in range(1, num_cols + 1):
        fig.update_xaxes(tickangle=45, row=i, col=j)
        fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Входящий поток иностранных студентов по группам стран с 2017 по 2022 г.png")
fig.show()


На основании отчетов inbound в разрезе весь мир -страна

In [101]:
# total_inbound = national[national['indicator_id'].isin([26637])]
# total_inbound['from'] = 'World'
# total_inbound.head()

<h4>Исходящие по странам</h4>

OE.5T8.40510 
Total outbound internationally mobile tertiary students studying abroad, all countries, both sexes (UIS estimate) (number)

In [31]:
outbound = pd.merge(countries.groupby(['from'])['2017'].sum().reset_index(),
                 countries.groupby(['from'])['2022'].sum().reset_index(),
                 on = 'from',
                 how='left')
outbound = outbound[~(outbound['2017'].isna()) & ~(outbound['2022'].isna())]
outbound = outbound[~(outbound['2017']==0) & ~(outbound['2022']==0)]
outbound['2022/2017,%'] = round(outbound['2022']/outbound['2017']*100,0).astype(int)-100
outbound['2022/2017,%'] = outbound['2022/2017,%'].apply(lambda x:'+'+str(x) if x>0 else str(x))
outbound = outbound[outbound['2022']>5000]
def group(x):
    if x<20000:
        group_name = '5-20 тыс.'
    elif 20000<=x<50000:
        group_name = '20-50 тыс.'
    elif 50000<=x<100000:
        group_name = '50-100 тыс.'
    elif 100000<=x<200000:
        group_name = '100-200 тыс.'
    else:
        group_name = 'Больше 200 тыс.'
    return group_name
outbound['group'] = outbound['2022'].apply(lambda x: group(x))
outbound.set_index('from', inplace=True)
outbound = outbound.sort_values(by='2022',ascending=False)
outbound.head()

,2017,2022,"2022/2017,%",group
from,,,,
China,839754.0,960174.0,+14,Больше 200 тыс.
India,350574.0,733082.0,+109,Больше 200 тыс.
Viet Nam,104888.0,145221.0,+38,100-200 тыс.
Uzbekistan,34689.0,140963.0,+306,100-200 тыс.
Germany,122284.0,129970.0,+6,100-200 тыс.


In [159]:
n_cols = 2  # Number of columns in subplot grid
# unique_groups  = diff['group'].unique()
unique_groups = ['Больше 200 тыс.','100-200 тыс.', '50-100 тыс.', '20-50 тыс.','5-20 тыс.']
num_plots = len(unique_groups)
num_cols = 2  # Maximum of 2 columns
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=unique_groups)
outbound = outbound.sort_values(by=['2022'],ascending=False)
for i, group in enumerate(unique_groups):
    filtered_df = outbound[outbound['group'] == group]
    row_index = i // n_cols + 1
    col_index = i % n_cols + 1
    
    trace1 = go.Bar(x=filtered_df.index,
        y=filtered_df['2017'],
        name=f'2017 - Group {group}',
        text=[f"{value/1000:.1f}k" if value < 1000 else f"{int(value/1000)}k" for value in filtered_df['2017']],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=12),
        marker_color='brown',
        marker_line=dict(width=3, color=['black' if country == 'Russia' else 'brown' for country in filtered_df.index]) 
        )
    trace2 = go.Bar(x=filtered_df.index,
        y=filtered_df['2022'],
        name=f'2022 - Group {group}',
        text=[f"{value/1000:.1f}k<br>({filtered_df['2022/2017,%'].values[j]}%)" if value < 1000 else f"{int(value/1000)}k<br>({filtered_df['2022/2017,%'].values[j]}%)" for j, value in enumerate(filtered_df['2022'])],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=12),
        marker_color='orange',
        marker_line=dict(width=3, color=['black' if country == 'Russia' else 'orange' for country in filtered_df.index]) 
        )
    
    fig.add_trace(trace1, row=row_index, col=col_index)
    fig.add_trace(trace2, row=row_index, col=col_index)
        
fig.update_layout(
    title='Исходящий поток иностранных студентов по группам стран с 2017 по 2022 г.',
    # xaxis_title='Country',
    yaxis_title='Число студентов, тыс.',
    barmode='group',  # Group bars together
    height=1600,
    width=2600,
    plot_bgcolor='white',
    annotations=[
                dict(
                    x=0.4,  # Center the annotation
                    y=-0.18,  # Move it below the chart
                    text="@Conspect Labs, Sources: UNESCO, OpenDoorsData, Nuffic, IIE, Statista, MoE of Kazakhstan, Iran, Egypt, Thailand",
                    showarrow=False,
                    xref="paper",  # Use paper coordinates
                    yref="paper",
                    font=dict(size=12, color="grey"),
                    align="left"
            )
            ]
)
for i in range(1, num_rows + 1):
    for j in range(1, num_cols + 1):
        fig.update_xaxes(tickangle=45, row=i, col=j)
        fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Исходящий поток иностранных студентов по группам стран с 2017 по 2022 г.png")
fig.show()

<h4>Student flows inside CIS</h4>

In [33]:
countries2 = countries[~(countries['2017'].isna()) & ~(countries['2022'].isna())]
countries2 = countries2[~(countries2['2017']==0) & ~(countries2['2022']==0)]

#some tweaks for Sankey diagram that demands different names for sources and targets
countries2['from'] = countries2['from']+ ' Out'
countries2['to'] = countries2['to']+ ' In'
countries2.head(2)

year,from,to,2017,2022
2,Afghanistan Out,Argentina In,15.0,16.0
4,Afghanistan Out,Australia In,44.0,73.0


In [34]:
cis_c = ['Russia','Moldova','Belarus','Kazakhstan','Uzbekistan',
          'Kyrgyzstan','Tajikistan','Turkmenistan',
          'Armenia','Azerbaijan','Ukraine']
cis = [item + ' Out' for item in cis_c] + [item + ' In' for item in cis_c]
cis_countries_from = countries2[countries2['from'].isin(cis)]
cis_countries_from.head(2)

year,from,to,2017,2022
953,Armenia Out,Argentina In,31.0,65.0
955,Armenia Out,Austria In,60.0,66.0


In [36]:
#Extract top 5 target countries for every source country in 2017 and 2022 
cis_countries_from_2022_top10 = cis_countries_from.groupby('from').apply(lambda x: x.nlargest(5, '2022')).reset_index(drop=True)
cis_countries_from_2022_top10.drop(columns='2017',inplace=True)
cis_countries_from_2017_top10 = cis_countries_from.groupby('from').apply(lambda x: x.nlargest(5, '2017')).reset_index(drop=True)
cis_countries_from_2017_top10.drop(columns='2022',inplace=True)
cis_countries_from_top = pd.concat([cis_countries_from_2017_top10,cis_countries_from_2022_top10])
cis_countries_from_top['to'].unique()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_20200\589063051.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

C:\Users\yuliy\AppData\Local\Temp\ipykernel_20200\589063051.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



array(['Russia In', 'France In', 'Germany In', 'USA In',
       'United Kingdom In', 'Turkey In', 'Georgia In', 'Poland In',
       'Lithuania In', 'Czechia In', 'China In', 'Kyrgyzstan In',
       'Kazakhstan In', 'Saudi Arabia In', 'Romania In', 'Italy In',
       'Belarus In', 'Malaysia In', 'Republic of Korea In', 'Latvia In',
       'Bulgaria In', 'Uzbekistan In', 'Slovakia In'], dtype=object)

In [37]:
#Draw Sankey for every year
c_a = ('Kazakhstan Out','Kyrgyzstan Out','Tajikistan Out','Turkmenistan Out','Uzbekistan Out')
e_c = ('Russia Out','Moldova Out','Belarus Out','Ukraine Out','Armenia Out','Azerbaijan Out')
regions = {c_a:'Central Asia',
           e_c:'Europe & Caucasus'}

for key,value in regions.items():
    print(value)
    df = cis_countries_from_top[cis_countries_from_top['from'].isin(key)]
    unique_labels = list(set(df['from'].tolist() + df['to'].tolist()))
    other_color = 'rgba(189, 195, 199, 0.8)'
    country_colors = {
        'Armenia': 'rgba(31, 119, 180, 0.8)',  # Blue
        'Azerbaijan': 'rgba(255, 127, 14, 0.8)',  # Orange
        'Belarus': 'rgba(44, 162, 44, 0.8)',  # Green
        'Kazakhstan': 'rgba(214, 39, 40, 0.8)',  # Red
        'Kyrgyzstan': 'rgba(148, 103, 189, 0.8)',  # Purple
        'Moldova': 'rgba(140, 86, 75, 0.8)',  # Brown
        'Russia': 'rgba(227, 119, 194, 0.8)',  # Pink
        'Tajikistan': 'rgba(127, 127, 127, 0.8)',  # Gray
        'Turkmenistan': 'rgba(188, 189, 34, 0.8)',  # Yellow-green
        'Ukraine': 'rgba(23, 190, 239, 0.8)',  # Cyan
        'Uzbekistan': 'rgba(255, 187, 120, 0.8)',   # Light Orange (to make it distinct)
        'France':other_color, 
        'Germany':other_color, 
        'USA':other_color, 
        'United Kingdom':other_color, 
        'Turkey':other_color, 
        'Georgia':other_color, 
        'Poland':other_color, 
        'Lithuania':other_color, 
        'Romania':other_color, 
        'Italy':other_color, 
        'Czechia':other_color, 
        'Republic of Korea':other_color,
        'Saudi Arabia':other_color,
        'Latvia':other_color,
        'Malaysia':other_color,
        'Bulgaria':other_color,
        'Slovakia':other_color,
        'China':other_color,
        'India':'rgba(97, 126, 79, 0.8)',
        'Egypt':'rgba(97, 60, 79, 0.8)'
    }

    # You can use color mapping for countries. Change i and coeffs to get distinct colors.
    # country_colors = {country: f'rgba({(i+100) * 20 % 255}, {((i+50) * 200 + 100) % 255}, {((i+200) * 380) % 255}, 1)'
    #                   for i, country in enumerate(set([label.split()[0] for label in unique_labels]))}
    # Function to get sorted unique labels based on total link values

    def get_sorted_labels(df, year):
        link_sums = pd.concat([
            df.groupby('from')[year].sum(),
            df.groupby('to')[year].sum()
        ]).groupby(level=0).sum()
        return link_sums.sort_values(ascending=False).index.tolist()

    # Function to build a Sankey trace for a specific year
    def build_sankey_trace(df, year,pad_size):
        sorted_labels = get_sorted_labels(df, year)
        unique_labels_sorted = [label for label in unique_labels if label in sorted_labels]
        
        # Map source and target to indices
        source_indices = [unique_labels_sorted.index(src) for src in df['from']]
        target_indices = [unique_labels_sorted.index(tgt) for tgt in df['to']]
        
        # Assign link colors based on source country
        link_colors = [country_colors[" ".join(src.split()[:-1])].replace('1)', '0.8)') for src in df['from']]
        
        return go.Sankey(
            node=dict(
                pad=pad_size,
                thickness=50,
                line=dict(color="black", width=0.5),
                label=[f"{label} ({int(df[df['from'] == label][year].sum() + df[df['to'] == label][year].sum())})"
                    for label in unique_labels_sorted],
                color=[country_colors[" ".join(label.split()[:-1])] for label in unique_labels_sorted]
            ),
            link=dict(
                source=source_indices,
                target=target_indices,
                value=df[year],
                color=link_colors,
                label=[f"{df.iloc[i]['from']} → {df.iloc[i]['to']}: {df.iloc[i][year]}"
                    for i in range(len(df))]
            )
        )

    def generate_annotations(df, unique_labels_sorted, year):
        annotations = []
        df=df[['from','to',year]].dropna()
        for i, row in df.iterrows():
            source_idx = unique_labels_sorted.index(row['from'])
            target_idx = unique_labels_sorted.index(row['to'])
            annotations.append(dict(
                # x=x_pos, y=y_pos,
                text=f"{int(row[year])}",  # Display value as an integer
                showarrow=False,
                font=dict(size=10, color="black"),
                align="center"
            ))
        return annotations

    fig = make_subplots(
        rows=1, cols=2, 
        specs=[[{'type': 'domain'}, {'type': 'domain'}]]  # Specify 'domain' for Sankey
    )
    fig.add_trace(build_sankey_trace(df, "2017",48), row=1, col=1)
    fig.add_trace(build_sankey_trace(df, "2022",18), row=1, col=2)
    fig.update_layout(
        title_text=f"Students Flows from {value} CIS Countries, Top 5 destinations for every country",
        font=dict(size=12, color='black', family='Times'),
        height=600,
        width=1000,
        plot_bgcolor='white',
        annotations=[
            dict(
                x=0.2,  # Center of first subplot
                y=1.05, 
                text="2017",
                showarrow=False,
                font=dict(size=16, color="black")
            ),
            dict(
                x=0.78,  # Center of second subplot
                y=1.05, 
                text="2022",
                showarrow=False,
                font=dict(size=16, color="black"),
            ),
            dict(
                x=0.002,  # Comments
                y=-0.1, 
                text="@Conspect Labs, Sources: UNESCO, OpenDoorsData, Nuffic, IIE, Statista, MoE of Kazakhstan, Iran, Egypt, Thailand",
                showarrow=False,
                font=dict(size=12, color="black")
            )
        ]
    )

    output_file = f"top_5_destinations_sankey_diagrams _{value}.png"  # You can change this to .png, .pdf, or .jpeg
    pio.write_image(fig, output_file, format="png", width=1600, height=800)
    # Show the figure
    fig.show()

Central Asia


Europe & Caucasus


<h4>Top 20 source countries for Russia</h4>

In [38]:
#Extract all source countries for Russia
inbound_country='Russia In'
inbound_to_country = countries2[countries2['to']==inbound_country]
inbound_to_country.head()

year,from,to,2017,2022
91,Afghanistan Out,Russia In,188.0,1602.0
210,Albania Out,Russia In,4.0,92.0
329,Algeria Out,Russia In,246.0,1524.0
567,Angola Out,Russia In,865.0,805.0
923,Argentina Out,Russia In,6.0,46.0


In [161]:
#Extract top 20 source countries for Russia in 2017 and 2022 
inbound_to_country_from_2017_top20 = inbound_to_country.sort_values(by='2017', ascending=False).head(20)
inbound_to_country_from_2017_top20 = inbound_to_country_from_2017_top20[['from','to','2017']]
inbound_to_country_from_2017_top20['from'] = inbound_to_country_from_2017_top20['from'].apply(lambda x:x.replace(' Out',''))
inbound_to_country_from_2017_top20 = inbound_to_country_from_2017_top20[['from','2017']].set_index('from').reset_index()
inbound_to_country_from_2017_top20['color'] = inbound_to_country_from_2017_top20['from'].map(country_colors)

inbound_to_country_from_2017_top20


year,from,2017,color
0,Kazakhstan,65237.0,"rgba(214, 39, 40, 0.8)"
1,Uzbekistan,20862.0,"rgba(255, 187, 120, 0.8)"
2,Turkmenistan,17457.0,"rgba(188, 189, 34, 0.8)"
3,Ukraine,15263.0,"rgba(23, 190, 239, 0.8)"
4,Tajikistan,14204.0,"rgba(127, 127, 127, 0.8)"
5,China,11950.0,"rgba(189, 195, 199, 0.8)"
6,Belarus,11600.0,"rgba(44, 162, 44, 0.8)"
7,Azerbaijan,11269.0,"rgba(255, 127, 14, 0.8)"
8,India,6544.0,"rgba(97, 126, 79, 0.8)"
9,Kyrgyzstan,5523.0,"rgba(148, 103, 189, 0.8)"


In [162]:
inbound_to_country_from_2022_top20 = inbound_to_country.sort_values(by='2022', ascending=False).head(20)
inbound_to_country_from_2022_top20 = inbound_to_country_from_2022_top20[['from','to','2022']]
inbound_to_country_from_2022_top20['from'] = inbound_to_country_from_2022_top20['from'].apply(lambda x:x.replace(' Out',''))
inbound_to_country_from_2022_top20 = inbound_to_country_from_2022_top20[['from','2022']].set_index('from').reset_index()
inbound_to_country_from_2022_top20['color'] = inbound_to_country_from_2022_top20['from'].map(country_colors)
inbound_to_country_from_2022_top20

year,from,2022,color
0,Kazakhstan,53935.0,"rgba(214, 39, 40, 0.8)"
1,Uzbekistan,48430.0,"rgba(255, 187, 120, 0.8)"
2,China,40797.0,"rgba(189, 195, 199, 0.8)"
3,Turkmenistan,27095.0,"rgba(188, 189, 34, 0.8)"
4,Tajikistan,27086.0,"rgba(127, 127, 127, 0.8)"
5,India,19734.0,"rgba(97, 126, 79, 0.8)"
6,Egypt,15616.0,"rgba(97, 60, 79, 0.8)"
7,Belarus,9821.0,"rgba(44, 162, 44, 0.8)"
8,Ukraine,9446.0,"rgba(23, 190, 239, 0.8)"
9,Kyrgyzstan,9345.0,"rgba(148, 103, 189, 0.8)"


In [164]:
inbound_to_country_from_2017_top20.sort_values(by='2017',ascending=True,inplace=True)
inbound_to_country_from_2022_top20.sort_values(by='2022',ascending=True,inplace=True)

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['2017', '2022'],
    row_heights =[1]
)
trace1 = go.Bar(
    y=inbound_to_country_from_2017_top20['from'],
    x=inbound_to_country_from_2017_top20['2017'],
    orientation='h',
    name=f'2017',
    text=[f"{value/1000:.1f}k" if value < 2000 else f"{int(value/1000)}k" for value in inbound_to_country_from_2017_top20['2017']],  # Value labels in thousands
    textposition='auto',
    textfont=dict(size=12),
    marker_color=[color for color in inbound_to_country_from_2017_top20['color']]
    )
trace2 = go.Bar(
    y=inbound_to_country_from_2022_top20['from'],
    x=inbound_to_country_from_2022_top20['2022'],
    orientation='h',
    name=f'2022',
    text=[f"{value/1000:.1f}k" if value < 2000 else f"{int(value/1000)}k" for value in inbound_to_country_from_2022_top20['2022']],  # Value labels in thousands
    textposition='auto',
    textfont=dict(size=12),
    marker_color=[color for color in inbound_to_country_from_2022_top20['color']]
    )
    
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
        
fig.update_layout(
    title='Топ-20 стран по количеству студентов в России в 2017 и 2022 г.png',
    yaxis_title='Число студентов',
    barmode='group',  # Group bars together
    height=600,
    width=1000,

    margin=dict(l=50, r=50, t=50, b=100),
    plot_bgcolor='white',
    annotations=[
            dict(
                x=0.4,  # Center the annotation
                y=-0.18,  # Move it below the chart
                text="@Conspect Labs, Sources: UNESCO, OpenDoorsData, Nuffic, IIE, Statista, MoE of Kazakhstan, Iran, Egypt, Thailand",
                showarrow=False,
                xref="paper",  # Use paper coordinates
                yref="paper",
                font=dict(size=12, color="grey"),
                align="left"
        )
        ]
)

pio.write_image(fig, "Топ-20 стран по количеству студентов в России в 2017 и 2022 гг.png")
fig.show()

Российские вузы зависят в основном от стран Центральной Азии и Китая - 60% студентов приезжают из Казахстана, Узбекистана, Туркменистана, Таджикистана и Китая. Диверсификация рынка происходит, но не быстро: в 2017 году топ-10 стран-доноров России составляли 85% въезда студентов, в 2022-м - 78%. 

In [ ]:
print(inbound_to_country_from_2017_top20['2017'].sum()/)

In [696]:
inbound_to_country.sort_values(by='2017', ascending=False).head(5)['2017'].sum()/inbound_to_country['2017'].sum()

np.float64(0.6293490911499484)

In [ ]:
inbound_to_country.sort_values(by='2022', ascending=False).head(5)['2022'].sum()/inbound_to_country['2022'].sum()

np.float64(0.5899888186650563)

In [697]:
inbound_to_country.sort_values(by='2017', ascending=False).head(10)['2017'].sum()/inbound_to_country['2017'].sum()

np.float64(0.8511728470993443)

In [695]:
inbound_to_country.sort_values(by='2022', ascending=False).head(10)['2022'].sum()/inbound_to_country['2022'].sum()

np.float64(0.7812135634974259)

<h4>Роль России в выездном потоке по странам в 2017 и 2022 гг</h4>

In [116]:
results = []
for every in countries2['from'].unique():
    source = countries2[countries2['from']==every]
    source_sum_17 = countries2[countries2['from']==every]['2017'].sum()
    russia_in_17 = source[source['to']=='Russia In']['2017'].sum()
    share_17 = round(russia_in_17/source_sum_17*100,1)
    source_sum_22 = countries2[countries2['from']==every]['2022'].sum()
    russia_in_22 = source[source['to']=='Russia In']['2022'].sum()
    share_22 = round(russia_in_22/source_sum_22*100,1)
    # print(every, source_sum,russia_in,share)
    results.append({
        'Country': every,
        'Total_2017': source_sum_17,
        'Russia_In_2017': russia_in_17,
        'Share_to_Russia_%_2017': share_17,
        'Total_2022': source_sum_22,
        'Russia_In_2022': russia_in_22,
        'Share_to_Russia_%_2022': share_22,
    })
results_df = pd.DataFrame(results)
results_df['total_change'] = round((results_df['Total_2022']/results_df['Total_2017']-1)*100,1)
results_df['Russia_in_change'] = round((results_df['Russia_In_2022']/results_df['Russia_In_2017']-1)*100,1)
# results_df = results_df[results_df['Share_to_Russia_%']>1]
results_df = results_df[results_df['Russia_In_2022']!=0]
results_df = results_df[results_df['Share_to_Russia_%_2022']>=2]
results_df = results_df[results_df['Russia_In_2022']>=500]
results_df = results_df.reset_index()
results_df = results_df.drop(columns=['index']).sort_values(by='Share_to_Russia_%_2022',ascending=False)
results_df

,Country,Total_2017,Russia_In_2017,Share_to_Russia_%_2017,Total_2022,Russia_In_2022,Share_to_Russia_%_2022,total_change,Russia_in_change
31,Tajikistan Out,19156.0,14204.0,74.1,31553.0,27086.0,85.8,64.7,90.7
21,Kazakhstan Out,95570.0,65237.0,68.3,92404.0,53935.0,58.4,-3.3,-17.3
22,Kyrgyzstan Out,11153.0,5523.0,49.5,16326.0,9345.0,57.2,46.4,69.2
33,Turkmenistan Out,38924.0,17457.0,44.8,54259.0,27095.0,49.9,39.4,55.2
3,Armenia Out,5710.0,2720.0,47.6,5116.0,2424.0,47.4,-10.4,-10.9
5,Belarus Out,21877.0,11600.0,53.0,26364.0,9821.0,37.3,20.5,-15.3
35,Uzbekistan Out,32570.0,20862.0,64.1,132936.0,48430.0,36.4,308.2,132.1
12,Egypt Out,27475.0,660.0,2.4,61356.0,15616.0,25.5,123.3,2266.1
25,Moldova Out,17729.0,3402.0,19.2,19188.0,3360.0,17.5,8.2,-1.2
23,Latvia Out,4678.0,452.0,9.7,4376.0,697.0,15.9,-6.5,54.2


In [160]:
#вариант 1 - график изменения доли рынка России в выездном потоке стран-доноров
fig = go.Figure()
trace1 = go.Bar(
    x=[country.replace(' Out', '') for country in results_df['Country']],
    y=results_df['Share_to_Russia_%_2017'],
    name='2017',
    text=[f"{round(value, 1)}%" if value < 5 else f"{round(value)}%" for value in results_df['Share_to_Russia_%_2017']],  # Value labels in thousands
    textposition='auto',
    textfont=dict(size=12),
    marker_color='rgba(19, 17, 18, 1)'
    )
trace2 = go.Bar(
    x=[country.replace(' Out', '') for country in results_df['Country']],
    y=results_df['Share_to_Russia_%_2022'],
    name='2022',
    text=[f"{round(value, 1)}%" if value < 5 else f"{round(value)}%" for value in results_df['Share_to_Russia_%_2022']],  # Value labels in thousands
    textposition='auto',
    textfont=dict(size=16),
    marker_color="rgba(110, 74, 92, 1)"
    )
    
fig.add_trace(trace1)
fig.add_trace(trace2)    
fig.update_layout(
    title='Доля России в выездном потоке студентов по странам в 2017 и 2022 гг',
    yaxis_title='Число студентов',
    barmode='group',  # Group bars together
    height=600,
    width=1600,
    yaxis=dict(title='Доли рынка', showgrid=True),  # First subplot y-axis
    plot_bgcolor='white'
)

pio.write_image(fig, "Доля России в выездном потоке студентов по странам в 2017 и 2022 гг.png")
fig.show()

Наибольший вклад в прирост въезда за 2017-2022 годы внесли Китай, Египет, Индия, 

<h4>Куда едут студенты с основных мировых рынков?</h4>

Топ мировых рынков международного образования - Китай (более 900 тыс. студентов в год), Индия (более 700 тыс.), Вьетнам и Узбекистан (более 140 тыс. каждый), Германия и Франция (более 120 тыс.), Пакистан (115 тыс.),. Нигерия (107 тыс.), Непал, США и Сирия (более 90 тыс.)
<p> из них только в Узбекистане позиции России заметны. 

In [ ]:
top_out = ['India Out','China Out','Viet Nam Out','Uzbekistan Out','Germany Out','France','Pakistan Out','Nigeria Out','Nepal Out','USA Out',' Syrian Arab Republic Out']

In [166]:
#Where students from India went in 2017 and 2022
#Extract all source countries for Russia
otbound_country='Russia Out'
outbound_from_country = countries2[countries2['from']==otbound_country]
outbound_from_country.head()

year,from,to,2017,2022
18513,Russia Out,Argentina In,131.0,123.0
18514,Russia Out,Armenia In,1357.0,2082.0
18515,Russia Out,Australia In,737.0,477.0
18516,Russia Out,Austria In,1389.0,1792.0
18517,Russia Out,Azerbaijan In,441.0,477.0
